In [205]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from math import pi

from bokeh.io import show, output_file
from bokeh.layouts import row
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource

# Read csvs

These are the csv's we cleaned up and combined in step 01. 

In [206]:
ings = pd.read_csv('data/ings.csv', encoding = "ISO-8859-1", index_col=0)
#prods = prods.drop_duplicates(subset = 'product')
prods = pd.read_csv('data/prods.csv', encoding = "ISO-8859-1", index_col=0)
#prods = prods.drop_duplicates(subset = 'product')
prod_ing = pd.read_csv('data/prod_ing.csv', encoding = "ISO-8859-1", index_col=0)
prod_ing = prod_ing.drop(['diff'], axis = 1)
prod_ing = prod_ing.sort_values(['id', 'order'])

print('Number of products: ', len(prods))
print('Number of unique ingredients: ', len(ings))
print('Number of total ingredients: ', len(prod_ing))

Number of products:  15945
Number of unique ingredients:  7325
Number of total ingredients:  545617


### Ings
Ings df is just a database of unique ingredients and their harmful score, function and notes.

In [207]:
#caught a rogue ingredient
ings['ingredient'] = ings['ingredient'].replace('Stearic Acid(Masking, Fragrance, Emulsion Stabilising, Emulsifying, Sufactant, Refatting, Surfactantsurfactant-Cleansing Agent Is Included As A Function For The Soap Form Of Stearic Acid.', 'Stearic Acid')
ings = ings.drop_duplicates(subset = 'ingredient')
print('Number of unique ingredients: ', len(ings))
ings.head()

Number of unique ingredients:  7324


,id,ingredient,ewg,cir,func_Abrasive,func_Antimicrobial,func_Antioxidant,func_Antistatic Agent,func_Astringent,func_Binding,...,notes_Comedogenic Rating (3),notes_Comedogenic Rating (4),notes_Comedogenic Rating (5),notes_Good for Dry Skin,notes_Good for Oily Skin,notes_Good for Sensitive Skin,notes_Paraben,notes_Promotes Wound Healing,notes_Sulfate,notes_UV Protection
0,0,Water,1,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,Hydrogenated Polyisobutene,1,A,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,Butylene Glycol,1,A,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,Sesamum Indicum (Sesame) Seed Oil,1,A,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,Ammonium Acryloyldimethyltaurate/VP Copolymer,1,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [208]:
ing_uniqueID = ings.loc[:,['ingredient']].reset_index()
ing_uniqueID = ing_uniqueID.drop(['index'], axis = 1)
ing_uniqueID['uniqueID'] = ing_uniqueID.index
#ing_uniqueID

### Prods
Prods df is a database of all the products with various information including brand, price and rating

In [209]:
prods.head(3)

,id,product,brand,price,rating,ratingsCount,cat_Acne&BlemishTreatments,cat_Ampoules,cat_BodySkincare,cat_Cleansers,...,note_AlcoholFree,note_Anti-Aging,note_Brightening,note_ContainsAlcohol,note_ContainsParaben,note_ContainsSulfate,note_ParabenFree,note_PromotesWoundHealing,note_SulfateFree,note_UVProtection
0,0,Naturally Gentle Eye Makeup Remover,Clinique,20.00,5.0,16,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0
2,2,Foot Cream Norwegian Formula,Neutrogena,23.82,4.9,84,0,0,0,0,...,1,0,1,0,1,0,0,1,1,0
3,3,Essential Power Skin Refiner_Moisture,Laneige,28.20,5.0,12,0,0,0,0,...,0,0,1,1,0,0,1,1,1,0


### Prod_ing
Prod ing is a database that essentiall links prods and ings. it contains the product id, ingredient and ingredient order.

In [210]:
prod_ing.head(3)

,id,ingredient,order
0,0,Water,1
1,0,Hydrogenated Polyisobutene,2
2,0,Butylene Glycol,3


### Add a column that is a unique ingredient identifier

In [211]:
#id is product id
#uniqueID is ingredient ID
prod_ing = pd.merge(prod_ing, ing_uniqueID, on='ingredient')
prod_ing = prod_ing.sort_values(['id', 'order'])
prod_ing.head()

,id,ingredient,order,uniqueID
0,0,Water,1,0
15954,0,Hydrogenated Polyisobutene,2,1
16617,0,Butylene Glycol,3,2
24563,0,Sesamum Indicum (Sesame) Seed Oil,4,3
24889,0,Ammonium Acryloyldimethyltaurate/VP Copolymer,5,4


In [213]:
#reset index and drop
prod_ing = prod_ing.reset_index(drop=True)

## Reformatting Prod_ing
Instead of having each ingredient in it's own row, we want to create a list of ingredients for each product while retaining the order.

In [215]:
prod_ing.groupby('id')['ingredient'].apply(list)

id
0        [Water, Hydrogenated Polyisobutene, Butylene G...
1        [Mineral Oil, Glycerin, PEG-12, Cyclopentasilo...
2        [Water, Glycerin, Cetearyl Alcohol, Petrolatum...
3        [Water, Glycereth-26, Alcohol, Butylene Glycol...
4        [Petrolatum, Glycerin, Cocamidopropyl Betaine,...
5        [Water, Butylene Glycol, Glycerin, Disodium Su...
6        [Mineral Oil, Avena Sativa (Oat) Kernel Flour,...
7        [Sodium Palm Kernelate, Water, Sodium Borate, ...
8        [Water, Hamamelis Virginiana (Witch Hazel) Wat...
9        [Water, Caprylic/Capric Triglyceride, Glycerin...
10       [Water, Glycerin, Sodium Laureth Sulfate, PEG-...
11       [Water, Tri-C14-15 Alkyl Citrate, Cyclohexasil...
12       [Water, Butylene Glycol, Propylene Glycol, Dip...
13       [Sodium Palm Kernelate, Water, Petrolatum, Gly...
14       [Water, Chamaecyparis Obtusa Water, Glycerin, ...
15       [Snail Secretion Filtrate, Water, Butylene Gly...
16       [Water, Bentonite, Glycerin, Magnesium Alumi

In [216]:
#cleanup of a rogue ingredient here as well
prod_ing['ingredient'] = prod_ing['ingredient'].replace('Stearic Acid(Masking, Fragrance, Emulsion Stabilising, Emulsifying, Sufactant, Refatting, Surfactantsurfactant-Cleansing Agent Is Included As A Function For The Soap Form Of Stearic Acid.', 'Stearic Acid')

##group and create list
prod_ing_lists = prod_ing.groupby('id')['ingredient'].apply(list)
prod_ing_ID_lists = prod_ing.groupby('id')['uniqueID'].apply(list)

##convert back to dataframe and reset index
prod_ing_df = prod_ing_lists.to_frame().reset_index()
prod_ing_ID_df = prod_ing_ID_lists.to_frame().reset_index()
##check to make sure unique id's are still in tact (id shouldn't be exactly == to index)
prod_ing_lists.head()

id
0    [Water, Hydrogenated Polyisobutene, Butylene G...
1    [Mineral Oil, Glycerin, PEG-12, Cyclopentasilo...
2    [Water, Glycerin, Cetearyl Alcohol, Petrolatum...
3    [Water, Glycereth-26, Alcohol, Butylene Glycol...
4    [Petrolatum, Glycerin, Cocamidopropyl Betaine,...
Name: ingredient, dtype: object

In [217]:
prod_ing_df.tail()

,id,ingredient
19858,19922,"[Water, Dimethicone, Glycerin, Palmitic Acid(E..."
19859,19923,"[Water, Dipropylene Glycol, Caprylic/Capric Tr..."
19860,19924,"[Water, Polyvinyl Alcohol, Ethanolamine, 1,2-H..."
19861,19925,"[Ricinus Communis (Castor) Seed Oil, Lanolin, ..."
19862,19926,"[Water, Ascorbic Acid (Vitamin C, Glycerin, Di..."


In [218]:
prod_ing_lists = pd.merge(prod_ing_df, prod_ing_ID_df, on='id')
prod_ing_lists = prod_ing_lists.rename(columns = {'ingredient': 'ingList', 'uniqueID': 'ing#List' })
prod_ing_lists.head()

,id,ingList,ing#List
0,0,"[Water, Hydrogenated Polyisobutene, Butylene G...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,1,"[Mineral Oil, Glycerin, PEG-12, Cyclopentasilo...","[22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 3..."
2,2,"[Water, Glycerin, Cetearyl Alcohol, Petrolatum...","[0, 23, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55..."
3,3,"[Water, Glycereth-26, Alcohol, Butylene Glycol...","[0, 62, 63, 2, 64, 65, 66, 67, 68, 69, 70, 71,..."
4,4,"[Petrolatum, Glycerin, Cocamidopropyl Betaine,...","[47, 23, 92, 93, 94, 2, 95, 96, 97, 98, 99, 16..."


## Merge prod_ings and prods
We can now use the list to merge back with the products.

In [219]:
products_and_ingredients= pd.merge(prod_ing_lists, prods, on='id')
products_and_ingredients.tail(3)
len(products_and_ingredients)

15945

### Add column for number of ingredients

In [220]:
products_and_ingredients['ingCount'] = products_and_ingredients['ingList'].apply(lambda x: len(x))
products_and_ingredients.head(3)

,id,ingList,ing#List,product,brand,price,rating,ratingsCount,cat_Acne&BlemishTreatments,cat_Ampoules,...,note_Anti-Aging,note_Brightening,note_ContainsAlcohol,note_ContainsParaben,note_ContainsSulfate,note_ParabenFree,note_PromotesWoundHealing,note_SulfateFree,note_UVProtection,ingCount
0,0,"[Water, Hydrogenated Polyisobutene, Butylene G...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",Naturally Gentle Eye Makeup Remover,Clinique,20.00,5.0,16,0,0,...,0,0,0,1,0,0,0,1,0,22
1,2,"[Water, Glycerin, Cetearyl Alcohol, Petrolatum...","[0, 23, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55...",Foot Cream Norwegian Formula,Neutrogena,23.82,4.9,84,0,0,...,0,1,0,1,0,0,1,1,0,20
2,3,"[Water, Glycereth-26, Alcohol, Butylene Glycol...","[0, 62, 63, 2, 64, 65, 66, 67, 68, 69, 70, 71,...",Essential Power Skin Refiner_Moisture,Laneige,28.20,5.0,12,0,0,...,0,1,1,0,0,1,1,1,0,38


In [222]:
products_and_ingredients['ing#List'] = products_and_ingredients['ing#List'].apply(lambda x: str(x))
products_and_ingredients['ing#List'] = products_and_ingredients['ing#List'].str.replace(',', '')
products_and_ingredients['ing#List'] = products_and_ingredients['ing#List'].str.replace('[', '')
products_and_ingredients['ing#List'] = products_and_ingredients['ing#List'].str.replace(']', '')
products_and_ingredients = products_and_ingredients.drop_duplicates(subset = 'product')

products_and_ingredients.head()
len(products_and_ingredients)

15945

# Recommendation engine
Based on the one detailed [here](https://www.datacamp.com/community/tutorials/recommender-systems-python)

In [223]:
#Import TfIdfVectorizer from scikit-learn
#Convert a collection of raw documents to a matrix of TF-IDF features.
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object
tfidf = TfidfVectorizer()
tfidf = TfidfVectorizer(stop_words=['Water'])

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(products_and_ingredients['ing#List'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(15945, 6888)

In [224]:
tfidf_matrix

<15945x6888 sparse matrix of type '<class 'numpy.float64'>'
	with 421665 stored elements in Compressed Sparse Row format>

In [225]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [226]:
indices = pd.Series(products_and_ingredients.index, index=products_and_ingredients['product']).drop_duplicates()

In [227]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(product, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[product]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    product_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return products_and_ingredients[['product', 'ingList']].iloc[product_indices]

In [140]:
get_recommendations('Foot Cream Norwegian Formula')

,product,ingList
5468,Daily Serum,"[Water, Butylene Glycol, Glycerin, Keratin Ami..."
5799,Environmental Rescue Mask,"[Water, Butylene Glycol, Glycerin, Keratin Ami..."
7909,Gesichtswasser Soft & Clear,"[Water, Sodium Chloride, Glycerin, Sodium Hydr..."
14697,Private Formula Day Cream Moisturiser,"[Water, Sesamum Indicum (Sesame) Seed Oil, Dis..."
9567,Ultra-Gentle Facial Moisturizer (Oil Free),"[Water, Methylparaben, Ethylparaben, Propylpar..."
1165,Moisture Oil-Free for Sensitive Skin (Canadian...,"[Water, Methylparaben, Ethylparaben, Propylpar..."
1171,"Oil-Free Moisture, for Sensitive Skin","[Water, Methylparaben, Ethylparaben, Propylpar..."
7485,Oil-Free Moisturizing Facial Lotion,"[Water, Methylparaben, Ethylparaben, Propylpar..."
5020,Dual Treatment Moisture Lotion with SPF 15,"[Water, Methylparaben, Ethylparaben, Propylpar..."
1252,Oil-Free Moisture Sensitive Skin Ultra-Gentle ...,"[Water, Methylparaben, Ethylparaben, Propylpar..."


In [144]:
ingTest = products_and_ingredients[products_and_ingredients['product'].str.match('Foot Cream Norwegian Formula')]
ingTest = ingTest['ingList']
print(*ingTest, sep='\n')

['Water', 'Sodium Chloride', 'Methylparaben', 'Propylparaben', 'Glycerin', 'Cetearyl Alcohol', 'Petrolatum', 'Cyclomethicone', 'Panthenol', 'Dimethicone', 'Keratin Amino Acids', 'Menthol', 'Allantoin', 'Dilauryl Thiodipropionate', 'Alpha-Bisabolol', 'Tocopheryl Linoleate', 'Sodium Cetearyl Sulfate', 'Sodium Sulfate', 'Diazolidinyl Urea', 'Fragrance']


'\n'